In [1]:
from haystack.telemetry import tutorial_running
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
import logging
from haystack.nodes import PreProcessor
from langdetect import detect
import nltk
import pandas as pd



c:\Users\sotof\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Logs
logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [3]:
# Specify the file path to your CSV file
file_path = "dominio3Test.csv"

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df.head()

,PageURL,Title,CreationDate,Seccion,Subseccion,Contenido,WikipageID,LastModified
0,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906,2023-09-25T06:26:03Z
1,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,"Structure, bonding and molecular vibrations",The symmetry of a carbon dioxide molecule is l...,5906,2023-09-25T06:26:03Z
2,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,In aqueous solution,"Carbon dioxide is soluble in water, in which i...",5906,2023-09-25T06:26:03Z
3,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Chemical reactions of CO2,CO2 is a potent electrophile having an electro...,5906,2023-09-25T06:26:03Z
4,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,Physical properties,Carbon dioxide is colorless. At low concentrat...,5906,2023-09-25T06:26:03Z


In [4]:
# Extraer las primeras 'numero_de_filas_a_extraer' filas y almacenarlas en un nuevo DataFrame
df_test = df.head(3)

df_test

,PageURL,Title,CreationDate,Seccion,Subseccion,Contenido,WikipageID,LastModified
0,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,NaN,NaN,Carbon dioxide is a chemical compound with the...,5906,2023-09-25T06:26:03Z
1,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,"Structure, bonding and molecular vibrations",The symmetry of a carbon dioxide molecule is l...,5906,2023-09-25T06:26:03Z
2,https://en.wikipedia.org/wiki/Carbon_dioxide,Carbon dioxide,2023-09-25T06:26:03Z,Chemical and physical properties,In aqueous solution,"Carbon dioxide is soluble in water, in which i...",5906,2023-09-25T06:26:03Z


In [25]:
import io

# Specify the row and column you want to extract
row_index = 1  # Index of the row (e.g., 2 for 'Charlie')
column_name = 'Contenido'  # Name of the column


# Extract the cell content
cell_content = df_test.at[row_index, column_name]

# Create an in-memory file
memory_file = io.StringIO()

# Write text to the file
memory_file.write(cell_content)

# Get the contents of the file
# file_contents = memory_file.getvalue()

# Close the in-memory file (optional, but good practice)
# memory_file.close()

# Print the contents
#print(file_contents)

2009

In [31]:
import pandas as pd

# Especifica el índice de fila y la columna que deseas extraer
fila = 1  # Por ejemplo, extraeremos la fila 2 (índice 2)
columna = 'Contenido'

# Almacenar el contenido de la celda en una variable
contenido = df_test.at[fila, columna]

# Crear el archivo TXT y escribir el contenido
nombre_archivo = "haystack-division/ContenidoCelda.txt"
with open(nombre_archivo, 'w', encoding='UTF-8') as archivo:
    archivo.write(contenido)

print(f"El archivo {nombre_archivo} se ha creado y el contenido se ha almacenado correctamente.")



El archivo haystack-division/ContenidoCelda.txt se ha creado y el contenido se ha almacenado correctamente.


In [30]:

# Borrar el contenido del archivo (dejar el archivo vacío)
with open(nombre_archivo, 'w') as archivo:
    archivo.write("")

print(f"El contenido del archivo {nombre_archivo} se ha borrado correctamente.")

El contenido del archivo haystack-division/ContenidoCelda.txt se ha borrado correctamente.


In [38]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path="haystack-division\ContenidoCelda.txt", meta=None)[0]

In [33]:
from haystack.utils import convert_files_to_docs

doc_dir = 'haystack-division'

all_docs = convert_files_to_docs(dir_path=doc_dir)

INFO - haystack.utils.preprocessing -  Converting haystack-division\ContenidoCelda.txt


In [39]:
from haystack.nodes import PreProcessor


# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

preprocessor = PreProcessor(

    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=False,
    split_by= "sentence",
    split_length = 5,
    split_respect_sentence_boundary=False

)


In [41]:
docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")






Preprocessing: 100%|██████████| 1/1 [00:00<00:00, 94.92docs/s]

n_docs_input: 1
n_docs_output: 3


In [42]:
print(docs_default)

[<Document: {'content': 'The symmetry of a carbon dioxide molecule is linear and centrosymmetric at its equilibrium geometry. The length of the carbon-oxygen bond in carbon dioxide is 116.3 pm, noticeably shorter than the roughly 140-pm length of a typical single C–O bond, and shorter than most other C–O multiply bonded functional groups such as carbonyls. Since it is centrosymmetric, the molecule has no electric dipole moment. As a linear triatomic molecule, CO2 has four vibrational modes as shown in the diagram. In the symmetric and the antisymmetric stretching modes, the atoms move along the axis of the molecule. ', 'content_type': 'text', 'score': None, 'meta': {'_split_id': 0}, 'id_hash_keys': ['content'], 'embedding': None, 'id': '7d4e9570ea7aa7e35e23d6f9ba666240'}>, <Document: {'content': 'There are two bending modes, which are degenerate, meaning that they have the same frequency and same energy, because of the symmetry of the molecule. When a molecule touches a surface or touc